In [ ]:
import os
os.chdir(r"c:\Users\rohan\Documents\Flutter Projects\saas-subscription-analytics")
os.getcwd()

In [ ]:
import duckdb
con = duckdb.connect("saas.db", read_only=False)

# Drop old tables to avoid duplicates
con.execute("""
DROP TABLE IF EXISTS accounts_clean;
DROP TABLE IF EXISTS subscriptions_clean;
DROP TABLE IF EXISTS churn_clean;
DROP TABLE IF EXISTS feature_usage_clean;
DROP TABLE IF EXISTS support_tickets_clean;
""")

# Load cleaned CSVs
con.execute("""
CREATE TABLE accounts_clean AS 
    SELECT * FROM read_csv_auto('data/processed/accounts_cleaned.csv');

CREATE TABLE subscriptions_clean AS 
    SELECT * FROM read_csv_auto('data/processed/subscriptions_cleaned.csv');

CREATE TABLE churn_clean AS 
    SELECT * FROM read_csv_auto('data/processed/churn_events_cleaned.csv');

CREATE TABLE feature_usage_clean AS 
    SELECT * FROM read_csv_auto('data/processed/feature_usage_cleaned.csv');

CREATE TABLE support_tickets_clean AS 
    SELECT * FROM read_csv_auto('data/processed/support_tickets_cleaned.csv');
""")

# Verify
print(con.execute("SHOW TABLES;").fetchall())


In [ ]:
query = open("sql/21_signup_retention_analysis.sql").read()
df_signup_retention = con.execute(query).df()
df_signup_retention.head()


In [ ]:
query = open("sql/22_feature_adoption_analysis.sql").read()
df_feature_cohorts = con.execute(query).df()
df_feature_cohorts.head()


In [ ]:
query = open("sql/23_churn_analysis.sql").read()
df_churn_cohorts = con.execute(query).df()
df_churn_cohorts.head()


In [ ]:
ret_matrix = df_signup_retention.pivot(
    index="cohort_month",
    columns="active_month",
    values="retention_rate"
)
ret_matrix


In [ ]:
adopt_matrix = df_feature_cohorts.pivot(
    index="cohort_month",
    columns="usage_month",
    values="adoption_rate"
)
adopt_matrix


In [ ]:
churn_matrix = df_churn_cohorts.pivot(
    index="cohort_month",
    columns="churn_month",
    values="churn_rate"
)
churn_matrix


In [ ]:
con.close()